# In this post I will show some good classifiers to approaching any natural language processing Classification Problem.

# Logistic Regression

# linearsvc

# Xgb

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from io import StringIO
from sklearn import model_selection 
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import plotly.plotly as py
import plotly.graph_objs as go
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
from collections import Counter
import xgboost as xgb
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape,test.shape

In [ ]:
sample.head()

In [ ]:
fig = plt.figure(figsize=(8,4))
train.groupby('author').count().plot.bar(ylim=0)
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS

# Thanks : https://www.kaggle.com/aashita/word-clouds-of-various-shapes ##
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), 
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'one', 'br', 'Po', 'th', 'sayi', 'fo', 'Unknown'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='black',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
    
plot_wordcloud(train['text'], title="Word Cloud of text in Train data")


In [ ]:
from wordcloud import WordCloud, STOPWORDS

# Thanks : https://www.kaggle.com/aashita/word-clouds-of-various-shapes ##
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), 
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'one', 'br', 'Po', 'th', 'sayi', 'fo', 'Unknown'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='black',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
    
plot_wordcloud(test['text'], title="Word Cloud of text in test data")

# The problem requires us to predict the author, i.e. EAP, HPL and MWS given the text. In simpler words, text classification with 3 different classes.

In [ ]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

# We use the LabelEncoder from scikit-learn to convert text labels to integers, 0, 1 2

In [ ]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=2019, 
                                                  test_size=0.1, shuffle=True)

In [ ]:
tfv = TfidfVectorizer(min_df=5,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

# we can se below how  the accuracy depend on regularization parameter of logistic regression.

In [ ]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
predictions

In [ ]:
idx = predictions.argmax(axis=1)
predictions = (idx[:,None] == np.arange(predictions.shape[1])).astype(int)
predictions

In [ ]:
predictions = [np.where(r == 1)[0][0] for r in predictions]

In [ ]:
 accuracy_score(yvalid, predictions)

In [ ]:
clf = LogisticRegression(C=3.0)
clf.fit(xtrain_tfv, ytrain)
predictions1 = clf.predict_proba(xvalid_tfv)
idx = predictions1.argmax(axis=1)
predictions1 = (idx[:,None] == np.arange(predictions1.shape[1])).astype(int)
predictions1 = [np.where(r == 1)[0][0] for r in predictions1]
accuracy_score(yvalid, predictions1)

# linearsvc

In [ ]:
clf = LinearSVC(max_iter=1000)
clf.fit(xtrain_tfv, ytrain)
y_pred = clf.predict(xvalid_tfv)
y_pred

In [ ]:
accuracy_score(yvalid, y_pred)

# Xgb

In [ ]:
param = {'objective':'multi:softprob',
          'num_class':3,
          'eval_metric':'merror',
           'nthread':10
         }

In [ ]:
dtrain = xgb.DMatrix(data=xtrain_tfv, label= ytrain)
dtest = xgb.DMatrix(data=xvalid_tfv)

In [ ]:
watchlist  = [(dtrain,'train')]
num_round = 2500
early_stopping_rounds=50
bst = xgb.train(param, dtrain, num_round, watchlist,early_stopping_rounds=early_stopping_rounds)

In [ ]:
y_pred = bst.predict(dtest)

In [ ]:
y_pred

In [ ]:
idx = y_pred.argmax(axis=1)
y_pred = (idx[:,None] == np.arange(y_pred.shape[1])).astype(int)
y_pred

In [ ]:
y_pred = [np.where(r == 1)[0][0] for r in y_pred]

In [ ]:
accuracy_score(yvalid, y_pred)